In [1]:
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

from typing import Union, List

from Database.Database_Weaviate import Database_Weaviate
# from LLM.LLM_GGUF import LLM_GGUF as LLM
from LLM.LLM_Ollama import LLM_Ollama as LLM

from langchain_weaviate.vectorstores import WeaviateVectorStore

In [2]:
class RAG_Bot:
    def __init__(self, collection_names=['Uk', 'Wales', 'NothernIreland', 'Scotland'], text_splitter='SpaCy', embedding_model="SentenceTransformers"):
        """
        Initializes the RAG_Bot object.
        
        Args:
            collection_names (list, optional): A list of collection names. Defaults to ['Uk', 'Wales', 'NothernIreland', 'Scotland'].
        """
        self.vector_db = Database_Weaviate(collection_names=collection_names, text_splitter=text_splitter, embedding_model=embedding_model)
        self.llm = LLM()

    def add_text(self, collection_name, text, metadata=None):
        """
        Adds text data to a specified collection in the Weaviate database.
        
        Args:
            collection_name (str): The name of the collection in the database.
            text (str): The text data to be added.
            metadata (dict): Additional metadata associated with the text.
        """
        self.vector_db.vector_store = WeaviateVectorStore(
            client=self.vector_db.client,
            index_name=collection_name,
            text_key="text",
            embedding=self.vector_db.embeddings,
        )
        
        self.vector_db.add_text_to_db(
            collection_name=collection_name,
            text=text,
            metadata=metadata
        )

    def __collection_routing(self, query) -> Union[str, List[str], None]:
        """
        This function will route the query to the correct collection.
        
        Args:
            query (str): The query to be routed.

        Returns:
            Union[str, None]: The collection name or None if no collection matches the query.
        """

        def check_for_existence_of_collection_names(query:str, collection_names:List[str]=['Uk', 'Wales', 'NothernIreland', 'Scotland']) -> Union[str, None]:
            Existing_collection_names = []
            for collection_name in collection_names:
                if collection_name.lower() in query.lower():
                    Existing_collection_names.append(collection_name)
            if len(Existing_collection_names) > 0:
                return Existing_collection_names
            else:
                return None
            
        mentioned_collections = check_for_existence_of_collection_names(query)
        if mentioned_collections == None:
            return None
        elif mentioned_collections != None and mentioned_collections != [] and len(mentioned_collections) >= 1:
            return mentioned_collections

    def query(self, query:str, k:int=1, search_type='Hybrid', max_new_tokens=1000):
        """
        Performs a RAG query on the specified collection using the Saul LLM

        Args:
            query (str): The query to search for similar documents.
            k (int, optional): The number of documents to return. Defaults to 1.

        Returns:
            Response: The response from the LLM.
        """

        Collection_to_query_from = self.__collection_routing(query)
        print(f'Collection_to_query_from: {Collection_to_query_from}')

        if not isinstance(Collection_to_query_from, list) and Collection_to_query_from == None:
            print('There was no collection mentioned in the query. Kindly mention a collection name/s for the query to be executed.')

        elif isinstance(Collection_to_query_from, list):
            self.__query_all(query=query, k=k, collection_names=Collection_to_query_from, search_type=search_type, max_tokens=max_new_tokens)
        
    def __query_all(self, query, k=1, collection_names:List[str]=['Uk', 'Wales', 'Nothernireland', 'Scotland'], search_type='Hybrid', max_tokens=1000):
        """
        Performs a RAG query on multiple specified collections using the Saul LLM.
        
        Args:
            collection_name (str): The name of the collection in the database.
            query (str): The query to search for similar documents.
            k (int, optional): The number of documents to return. Defaults to 1.
        
        Returns:
            None
        
        Prints the similarity score and the content of the top k documents that match the query.
        """
        
        # Creating a WeaviateVectorStore for all counties one by one
        for collection_name in collection_names:
            #Validate existence of the collection itself first.
            Validity = self.is_collection_empty(collection_name)
            print(f'The Collection: {collection_name} is empty(0)/Not Empty(1): {Validity}')
            
            if not Validity:
                if search_type == 'Vector':
                    self.vector_db.vector_store = WeaviateVectorStore(
                        client=self.vector_db.client,
                        index_name=collection_name,
                        text_key="text",
                        embedding=self.vector_db.embeddings,
                    )
                
                    # Get the current WeaviateVectorStore
                    current_db = self.vector_db.vector_store
                    
                    # Create a retriever for the current database
                    retriever = current_db.as_retriever(
                        search_kwargs={"k": k})

                    # Function to format documents into a single context string
                    def format_docs(docs):
                        print(f'The retrieved documents are:')
                        for idx,doc in enumerate(docs):
                            print(f'{idx} - Content: {doc.page_content[:50]}... - MetaData: {doc.metadata}')
                        return "\n\n".join(doc.page_content for doc in docs)
                    
                    retrieved_docs = retriever.get_relevant_documents(query)
                    context = format_docs(retrieved_docs)
                    
                    response = self.llm.chat(context={context},
                                            query={query},
                                            max_new_tokens=max_tokens)
                    print(f'\n\nThe response is from the collection: {collection_name}\n')
                    for chunk in response:
                        print(chunk, end='', flush=True)
                    print(f'\nThe response has been generated above ^\n\n')

                elif search_type == 'Hybrid':
                    current_collection = self.vector_db.client.collections.get(collection_name)
                    responses = current_collection.query.hybrid(query=query,
                                                                vector=self.vector_db.embeddings.embed_query(query),
                                                                limit=k)
                    Text_Docs = []
                    Text_Meta_Datas = []
                    
                    for o in responses.objects: #output docs of the hybrid search
                        Text_Docs.append(o.properties['text'])
                        Text_Meta_Datas.append({k: v for k, v in o.properties.items() if k != 'text'})

                    def format_docs(docs):
                        print(f'The retrieved documents are:')
                        for idx,(doc,meta) in enumerate(zip(docs,Text_Meta_Datas)):
                            print(f'{idx} - Content: {doc[:50]}... - MetaData: {meta}')
                        return "\n\n".join(doc for doc in docs)

                    concat_docs = format_docs(Text_Docs)

                    response = self.llm.chat(context={concat_docs},
                                            query={query},
                                            max_new_tokens=max_tokens)
                    print(f'\n\nThe response is from the collection: {collection_name}\n')
                    for chunk in response:
                        print(chunk, end='', flush=True)
                    print(f'\nThe response has been generated above ^\n\n')
                    
    def is_collection_empty(self, collection_name: str) -> bool:
        current_client = self.vector_db.client.collections.get(collection_name)
        for doc in current_client.iterator():
            return False
        return True

    def get_list_of_all_docs(self, collection_name:Union[str, List[str]]=None) -> None:
        """
        Function to get the list of all documents in the specified collection.
        
        Args:
            collection_name (Union[str, List[str]], optional): The name of the collection in the database. Defaults to None.
        
        Returns:
            None
        """
        if isinstance(collection_name, list):
            for collection in collection_name:
                is_empty = self.is_collection_empty(collection)
                if not is_empty:
                    self.get_list_of_all_docs(collection)

        elif isinstance(collection_name, str):
            collection_existence_validity = self.is_collection_empty(collection_name)
            if not collection_existence_validity:
                print(f'The collection {collection_name} has the following documents:')
                current_client = self.vector_db.client.collections.get(collection_name)
                for item in current_client.iterator():
                    for idxKey,Key in enumerate(item.properties.keys()):
                        print(f'{Key}:  {item.properties[Key]}')
                print('\n\n')

In [3]:
collection_names = ['Uk', 'Wales', 'NothernIreland', 'Scotland']
bot = RAG_Bot(collection_names=collection_names, text_splitter='SpaCy', embedding_model="SentenceTransformers")

Cluster_URL: https://gwqaighsqp6gtrk8kwmnmg.c0.us-west3.gcp.weaviate.cloud, Cluster_API: 2xKmTTGUawzXYItFrMR9JgTjY2oRkfzHFLWN


I0000 00:00:1724086040.991266  166086 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache


Creating 4 Weaviate Clusters - Option 3
Country.capitalize(): Uk
The collection: Uk already exists in the Weaviate Cluster
Country.capitalize(): Wales
The collection: Wales already exists in the Weaviate Cluster
Country.capitalize(): Nothernireland
The collection: NothernIreland already exists in the Weaviate Cluster
Country.capitalize(): Scotland
The collection: Scotland already exists in the Weaviate Cluster


In [4]:
# bot.vector_db.delete_all_collections()

# Single Collection Queries

### To mention the countries, write any country name in the following manner:
- Uk
- Wales
- Scotland
- NothernIreland

In [5]:
bot.query(query='Define labor laws and limitations within NothernIreland?', k=3, search_type='Hybrid') #or Vector with a Capital V #change the query here with your text

Collection_to_query_from: ['NothernIreland']
The Collection: NothernIreland is empty(0)/Not Empty(1): False
The retrieved documents are:
0 - Content: STATUTORY INSTRUMENTS
2022

No. 1145
TERMS AND CON... - MetaData: {'legislationType': 'May contain legislation that applies to NothernIreland', 'legislation': 'UK Statutory Instruments', 'year': '2022', 'title': 'The Exclusivity Terms for Zero Hours Workers (Unenforceability and Redress) Regulations 2022.txt', 'country': 'NothernIreland'}
1 - Content: The Secretary of State must from time to time—
(a)... - MetaData: {'year': '2022', 'legislation': 'UK Draft Statutory Instruments', 'title': 'The Exclusivity Terms for Zero Hours Workers (Unenforceability and Redress) Regulations 2022\nSuperseded by 2022 No. 1145.txt', 'legislationType': 'May contain legislation that applies to NothernIreland', 'country': 'NothernIreland'}
2 - Content: Draft Regulations laid before Parliament under sec... - MetaData: {'title': 'The Exclusivity Terms for Zero

In [6]:
bot.query(query='Define labor laws and limitations within Wales?', k=2, search_type='Vector') #or Vector with a Capital V #change the query here with your text

Collection_to_query_from: ['Wales']
The Collection: Wales is empty(0)/Not Empty(1): False
The retrieved documents are:
0 - Content: STATUTORY INSTRUMENTS
2022

No. 1145
TERMS AND CON... - MetaData: {'year': '2022', 'legislation': 'UK Statutory Instruments', 'title': 'The Exclusivity Terms for Zero Hours Workers (Unenforceability and Redress) Regulations 2022.txt', 'legislationType': 'May contain legislation that applies to Wales', 'country': 'Wales'}
1 - Content: (8)
Regulations under subsection (7)—
(a)
may make... - MetaData: {'year': '2023', 'legislation': 'UK Public General Acts', 'title': 'Public Order Act 2023.txt', 'legislationType': 'May contain legislation that applies to Wales', 'country': 'Wales'}


The response is from the collection: Wales

Based on the provided document, which is the Exclusivity Terms for Zero Hours Workers (Unenforceability and Redress) Regulations 2022, I can answer your question as follows:

The document defines "Wales" in section 7(10) to include the 

In [10]:
bot.query(query='If I crash my car, damage a housing property and injure a person in Wales. What is the consequence?', k=3, search_type='Vector') #or Vector with a Capital V #change the query here with your text

Collection_to_query_from: ['Wales']
The Collection: Wales is empty(0)/Not Empty(1): False
The retrieved documents are:
0 - Content: Commencement Information
I1
Reg. 132 in force at 2... - MetaData: {'year': '2021', 'legislation': 'UK Statutory Instruments', 'title': 'The Space Industry Regulations 2021.txt', 'legislationType': 'May contain legislation that applies to Wales', 'country': 'Wales'}
1 - Content: (b)
any claim which includes a claim by a tenant o... - MetaData: {'legislationType': 'May contain legislation that applies to Wales', 'legislation': 'UK Statutory Instruments', 'year': '2023', 'title': 'The Civil Procedure (Amendment No. 2) Rules 2023.txt', 'country': 'Wales'}
2 - Content: 16I
Penalties and other liabilities
(1)


This sec... - MetaData: {'year': '2022', 'legislation': 'UK Public General Acts', 'title': 'Judicial Review and Courts Act 2022.txt', 'legislationType': 'May contain legislation that applies to Wales', 'country': 'Wales'}


The response is from the collec